In [1]:
import polars as pl
import pandas as pd

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#scanning in our parquet from s3
source = "s3://alexaevafernando/ready.parquet"
df = pd.read_parquet(source)

In [3]:
print(df.head())

                         timestamp      X      Y pixel_color  user  \
0 2023-07-22 15:56:57.877000+00:00   69.0 -294.0     #FF4500     0   
1 2023-07-20 14:42:00.196000+00:00 -481.0 -133.0     #3690EA     1   
2 2023-07-20 14:51:11.887000+00:00 -472.0 -290.0     #3690EA     1   
3 2023-07-20 14:56:28.979000+00:00 -494.0 -356.0     #3690EA     1   
4 2023-07-20 15:01:31.998000+00:00 -500.0  -59.0     #3690EA     1   

               time_diff  time_diff_min  
0                    NaT            NaN  
1                    NaT            NaN  
2 0 days 00:09:11.691000       9.183333  
3 0 days 00:05:17.092000       5.283333  
4 0 days 00:05:03.019000       5.050000  


In [4]:
#filtering out admins
tops = df.groupby('user').size().reset_index(name='count')

tops = tops.sort_values(by='count', ascending=False)
admin = tops.head()
admins = admin['user'].unique()
nonadmins = df[~df['user'].isin(admins)]

In [5]:
# identifying bots

nonadmins = nonadmins.sort_values(by=['user','timestamp'])

def detect_session_changes(df):
    session_starts = ((df['time_diff_min'] >21) | (df['user'] != df['user'].shift(1))).astype(int)
    df['session_id'] = session_starts.cumsum()
    return df

nonadmins = detect_session_changes(nonadmins)
                      

In [6]:
nonadmins.head()

,timestamp,X,Y,pixel_color,user,time_diff,time_diff_min,session_id
0,2023-07-22 15:56:57.877000+00:00,69.0,-294.0,#FF4500,0,NaT,NaN,1
1,2023-07-20 14:42:00.196000+00:00,-481.0,-133.0,#3690EA,1,NaT,NaN,2
2,2023-07-20 14:51:11.887000+00:00,-472.0,-290.0,#3690EA,1,0 days 00:09:11.691000,9.183333,2
3,2023-07-20 14:56:28.979000+00:00,-494.0,-356.0,#3690EA,1,0 days 00:05:17.092000,5.283333,2
4,2023-07-20 15:01:31.998000+00:00,-500.0,-59.0,#3690EA,1,0 days 00:05:03.019000,5.050000,2


In [7]:
#finding the pixel counts of each session
session_counts = nonadmins.groupby('session_id').size().sort_values(ascending=False).reset_index(name='count')

In [8]:
#merging counts back into dataframe
nonadmins = nonadmins.merge(session_counts, how = 'left', on='session_id')

In [17]:
nonadmins.head(20)

,timestamp,X,Y,pixel_color,user,time_diff,time_diff_min,session_id,count
0,2023-07-22 15:56:57.877000+00:00,69.0,-294.0,#FF4500,0,NaT,NaN,1,1
1,2023-07-20 14:42:00.196000+00:00,-481.0,-133.0,#3690EA,1,NaT,NaN,2,10
2,2023-07-20 14:51:11.887000+00:00,-472.0,-290.0,#3690EA,1,0 days 00:09:11.691000,9.183333,2,10
3,2023-07-20 14:56:28.979000+00:00,-494.0,-356.0,#3690EA,1,0 days 00:05:17.092000,5.283333,2,10
4,2023-07-20 15:01:31.998000+00:00,-500.0,-59.0,#3690EA,1,0 days 00:05:03.019000,5.050000,2,10
5,2023-07-20 15:06:44.267000+00:00,-484.0,-369.0,#3690EA,1,0 days 00:05:12.269000,5.200000,2,10
6,2023-07-20 15:12:04.802000+00:00,-471.0,-365.0,#3690EA,1,0 days 00:05:20.535000,5.333333,2,10
7,2023-07-20 15:17:15.082000+00:00,-405.0,-72.0,#FF4500,1,0 days 00:05:10.280000,5.166667,2,10
8,2023-07-20 15:22:29.066000+00:00,-425.0,-56.0,#FFFFFF,1,0 days 00:05:13.984000,5.216667,2,10
9,2023-07-20 15:27:44.762000+00:00,-414.0,-56.0,#FF4500,1,0 days 00:05:15.696000,5.250000,2,10


In [10]:
#filtering to just bots
botfiltercount = nonadmins['count'] >45
botsct = nonadmins[botfiltercount]

In [11]:
import s3fs

In [12]:
bucket = 'alexaevafernando'

file_name = 'botsct.csv'
s3_uri = f"s3://{bucket}/{file_name}"

fs = s3fs.S3FileSystem()

with fs.open(s3_uri, 'wb')as f:
    botsct.to_csv(f)